In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 100)
from datetime import datetime
# from flask import Flask
# from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase, MappedAsDataclass
import sqlalchemy as sa
from src.app.server import Flask, url_object, db
from src.app.models import *


app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = url_object
db.init_app(app)

## ANALYTICS

Continue at Assign - Section 4 video 103
Charge Quartile by Period

Stats - Charge & Paid

In [ ]:
with app.app_context():
    stmt1 = db.select(ClaimsPaid.charge_allowed, ClaimsPaid.charge_paid_ins, ClaimsPaid.deduct_copay)
    result = db.session.execute(stmt1).all()

df = pd.DataFrame(result)
df.describe()


In [ ]:
with app.app_context():
    table = db.select(PeriodMember.period, PeriodMember.mem_acct_id, PeriodMember.claims_member)
    table = db.session.execute(table).all()

member_stmt = pd.DataFrame(table)
member_stmt


CONSOLIDATEDCODES

In [17]:
# https://stackoverflow.com/questions/76568476/how-to-load-a-pandas-dataframe-from-orm-sqlalchemy-from-an-existing-database

with app.app_context():
    con_df = pd.read_sql_table('v_consolidated_codes', con=db.engine,)
    per_sum = pd.read_sql_table('v_period_summary', con=db.engine)
    specialty = pd.read_sql_table('t_specialty', con=db.engine)
    injury = pd.read_sql_table('t_injury_disease', con=db.engine)
    facility = pd.read_sql_table('t_facility', con=db.engine)

con_df.rename(columns={'charge_trans_date': 'date'}, inplace=True)
con_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70166 entries, 0 to 70165
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   claim_trans        70166 non-null  int64         
 1   date               70166 non-null  datetime64[ns]
 2   period             70166 non-null  int64         
 3   mem_acct_id        70166 non-null  int64         
 4   injury_disease_id  70166 non-null  int64         
 5   specialty_id       70166 non-null  int64         
 6   facility_id        70166 non-null  int64         
 7   charge_allowed     70166 non-null  int64         
 8   charge_paid_ins    70166 non-null  int64         
 9   deduct_copay       70166 non-null  int64         
dtypes: datetime64[ns](1), int64(9)
memory usage: 5.4 MB


In [3]:
per_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   period                          12 non-null     int64  
 1   day_count                       12 non-null     int64  
 2   claims_period_count             12 non-null     float64
 3   claims_daily_avg                12 non-null     float64
 4   claims_period_paid              12 non-null     float64
 5   claims_paid_daily_avg           12 non-null     float64
 6   claims_period_count_cum         12 non-null     float64
 7   claims_period_average_cum       12 non-null     float64
 8   claims_period_paid_cum          12 non-null     float64
 9   claims_period_paid_average_cum  12 non-null     float64
dtypes: float64(8), int64(2)
memory usage: 1.1 KB


In [18]:
injury.info()
specialty.info()
facility.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          18 non-null     int64 
 1   name        18 non-null     object
 2   category    18 non-null     object
 3   color_code  18 non-null     object
dtypes: int64(1), object(3)
memory usage: 704.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          25 non-null     int64 
 1   name        25 non-null     object
 2   category    25 non-null     object
 3   color_code  25 non-null     object
dtypes: int64(1), object(3)
memory usage: 928.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          6 non-nul

COLUMNS

In [ ]:
con_df.rename(columns=lambda x: x.upper())

In [ ]:
con_df.head()

In [ ]:
# con_df.set_index('claim_trans')
# con_df.reset_index(drop=True)

ISIN

In [ ]:
# mask = (con_df.period == 12) & (con_df.deduct_copay > 10)
# con_df[mask]
# mask = (con_df.period.eq(12)) & (con_df.deduct_copay.gt(10))
# con_df[mask]

# con_df[con_df.period.isin([1,3,4])]

# per_sum.loc[per_sum.period.isin([1,2])]
# con_df.loc[con_df.injury_disease_id.isin([102, 104])]

AGGREGATES

In [ ]:
charges = con_df.charge_allowed.sum()
charge_averages = con_df.charge_allowed.mean()
mem_count = con_df.mem_acct_id.nunique()
mem_ave = charges / mem_count
charges_percentile = con_df.charge_allowed.quantile([0.25, 0.5, 0.75])
charges_percentile2 = con_df.charge_allowed.iloc[:1000].quantile([.5])
injury_count = con_df.injury_disease_id.value_counts().sort_values()
injury_count2 = con_df.injury_disease_id.value_counts(normalize=True)
#
# charges
# charge_averages
# mem_count
# mem_ave
# charges_percentile
# charges_percentile2
# injury_count
# injury_count2


DATAFRAMES

In [ ]:
con_df.describe().round()

In [ ]:
# con_df.iloc[:4, 1:4]

# con_df.loc[:2, 'mem_acct_id']
# con_df.loc[:2, ['mem_acct_id']]
# con_df.loc[:2, 'mem_acct_id':'facility_id']

In [ ]:
new_df = con_df.loc[:50, ['period', 'charge_trans_date', 'mem_acct_id', 'charge_allowed']]
new_df["overhead"] = new_df.charge_allowed * .25
new_df['profit'] = (new_df.charge_allowed + new_df.overhead) * .05
new_df['revenue'] = new_df.charge_allowed + new_df.overhead + new_df.profit
new_df.revenue.sum()

In [ ]:
# new_df.loc[new_df['period'] == 2, ['mem_acct_id', 'revenue']].head()
# mask = new_df['mem_acct_id'].isin([239176, 229442]) & (new_df.revenue > 200)
# new_df.loc[mask, ['period', 'revenue']]

# (new_df['charge_allowed'] < 100).mean()

mask = (new_df['charge_allowed'] > 200) & (new_df['period'] == 1)
new_df.loc[mask, 'revenue'].count()


QUERY WITH DATES

In [ ]:
threshold = 300

# new_df.query("period == 1 and charge_allowed > @threshold")
# new_df.query("charge_trans_date.dt.month == 10")


SORTING

In [ ]:
new_df.sort_values(['period', 'charge_allowed'], ascending=[True, False])

JOINING - Section 10

In [23]:
table_x = con_df.loc[:, ['period', 'date', 'mem_acct_id', 'injury_disease_id', 'specialty_id',
                         'facility_id', 'charge_allowed']]
table_x.loc[:8, ]

,period,date,mem_acct_id,injury_disease_id,specialty_id,facility_id,charge_allowed
0,2,2019-11-18,239176,108,211,302,220
1,1,2019-10-11,363226,113,209,302,60
2,1,2019-10-11,363226,113,209,302,290
3,1,2019-10-11,363226,113,209,302,60
4,1,2019-10-11,363226,113,209,302,60
5,1,2019-10-11,363226,113,209,302,116
6,1,2019-10-11,363226,113,209,302,289
7,1,2019-10-11,363226,113,209,302,89
8,1,2019-10-11,363226,113,209,302,4528


In [21]:
categories = pd.concat([injury, specialty, facility], ignore_index=True)
categories

,id,name,category,color_code
0,101,Abnormal_Discoveries,injury_disease,#0000A0
1,102,Blood_Organs,injury_disease,#368BC1
2,103,Circulatory,injury_disease,#493D26
3,104,Congenital_Malform,injury_disease,#4E8975
4,105,Digestive,injury_disease,#566D7E
5,106,Endocrine_Hormone,injury_disease,#7BCCB5
6,107,Eye_Adnexa,injury_disease,#7D1B7E
7,108,Genitourinary,injury_disease,#848482
8,109,Infectious_Diseases,injury_disease,#8EEBEC
9,110,Injury_Poisoning,injury_disease,#A0CFEC


In [30]:
quant = table_x.charge_allowed.quantile(.75)
quant
table_x['top_charge'] = table_x['charge_allowed'] > quant
table_x.top_charge.sum()
table_x.top_charge.count()
(table_x.top_charge.sum() / table_x.top_charge.count()).round(2)

188.0

17448

70166

0.25

In [31]:
1000000 / 47.65

20986.35886673662

RACING BAR

In [ ]:
with app.app_context():
    icd_racing = db.select(InjuryDiseaseRacing.name, InjuryDiseaseRacing.period, InjuryDiseaseRacing.color_code,
                           InjuryDiseaseRacing.claim_count_ytd)
    icd_racing = db.session.execute(icd_racing)
    # for r in icd_racing:
    #     print(r)


    icd_racing_table = pd.DataFrame(icd_racing)
    # icd_racing_table


In [ ]:
with app.app_context():
    icd_racing = db.select(InjuryDiseaseRacing.name, InjuryDiseaseRacing.period, InjuryDiseaseRacing.color_code,
                           InjuryDiseaseRacing.claim_count_ytd)
    icd_racing = db.session.execute(icd_racing)
    # for r in icd_racing:
    #     print(r)

icd_racing_table = pd.DataFrame(icd_racing)

dict_keys = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
periods = [1,2,3,4,5,6,7,8,9,10,11,12]
n_frame = {}

for p, d in zip(periods, dict_keys):
    dataframe = icd_racing_table[(icd_racing_table['period']==p)]
    dataframe = dataframe.nlargest(n=5, columns=['claim_count_ytd'])
    dataframe = dataframe.sort_values(by=['period', 'claim_count_ytd'])

    n_frame[d] = dataframe

fig = go.Figure(
    data=[go.Bar(
        x=n_frame['one']['claim_count_ytd'],
        y=n_frame['one']['name'],
        orientation='h',
        text=n_frame['one']['claim_count_ytd'],
        # texttemplate='#{text:,0f}',
        textfont={'size':18},
        textposition='inside',
        insidetextanchor='middle',
        width=0.9,
        marker={'color': n_frame['one']['color_code']}
    )],
    layout = go.Layout(
        xaxis=dict(range=[0,8600], autorange=False, title=dict(text='claim_count', font=dict(size=18))),
        yaxis=dict(range=[-0.5, 5.5], autorange=False, tickfont=dict(size=14)),
        title=dict(text='Claims per Injury_Disease Category', font=dict(size=28), x=0.5, xanchor='center'),
        # Button
        updatemenus=[dict(
            type='buttons',
            buttons=[dict(label='Play',
                          method='animate',
                          args=[None,
                                {'frame': {'duration': 1000, 'redraw': True},
                                 "transition": {'duration': 250, 'easing': "linear"}}]
            )]
        )]
    ),
    frames=[
        go.Frame(
            data=[
                go.Bar(x=value['claim_count_ytd'], y=value['name'],
                       orientation='h',
                       text=value['claim_count_ytd'],
                       marker={'color':value['color_code']})
            ],
            layout=go.Layout(
                xaxis=dict(range=[0,8600], autorange=False),
                yaxis=dict(range=[-0.5, 5.5], autorange=False, tickfont=dict(size=14)),
                title=dict(text='Claims per Injury_Disease Category: '+str(value['period'].values[0]),
                           font=dict(size=28))
            )
        )
        for key, value in n_frame.items()
    ]
)
# import plotly.io as pio
fig
# print(n_frame)


CASHFLOW

CASHFLOW CHART 1 - Daily Cash Paid

In [ ]:
with app.app_context():
    period_chosen = 2

    cash_paid = db.session.execute(db.select(DailyClaims.charge_trans_date, DailyClaims.charges_paid)
                                   .where(DailyClaims.period == period_chosen))

    cash_paid_df = pd.DataFrame(cash_paid)
    # cash_paid_df
    px.bar(cash_paid_df, x='charge_trans_date', y='charges_paid')



In [ ]:
with app.app_context():

    cash_paid_period = db.session.execute(db.select(PeriodSummary.period, PeriodSummary.claims_period_paid))
    cash_paid_period_df = pd.DataFrame(cash_paid_period)
    # cash_paid_period_df['claims_period_paid'] = cash_paid_period_df['claims_period_paid'].astype(float)
    # cash_paid_period_df.info()
    # cash_paid_period_df
    cash_paid_period_fig = px.bar(cash_paid_period_df, x='period', y='claims_period_paid')
    # cash_paid_period_fig


## MEMBER PARTICIPATION

In [ ]:
with app.app_context():
    member_participation = db.session.execute(db.select(DailyMember.charge_trans_date, DailyMember.member_count)
                                              .where(DailyMember.period == period_chosen))

    member_participation_chart = pd.DataFrame(member_participation)
    # member_participation_chart




In [ ]:
with app.app_context():

    daily_member = db.session.execute(db.select(DailyMember.charge_trans_date, DailyMember.member_count)
                                              .where(DailyMember.period == period_chosen))
    daily_member_df = pd.DataFrame(daily_member)
    daily_member_chart = px.bar(daily_member_df, x='charge_trans_date', y='member_count')
    # daily_member_chart

In [ ]:
with app.app_context():
    claims_test = db.session.execute(db.select(sa.func.count(Claims.mem_acct_id.distinct()))).scalar()
    # daily_member_average = db.session.execute(db.select(sa.func.count(Claims.mem_acct_id.distinct))).scalar()
    print(claims_test)
    # for r in claims_test:
    #     print(r)



## CREATING DATA CLASS

In [ ]:
import pandas as pd
file_df= r'C:\Users\jchri\PycharmProjects\dash_website\src\app\data\corp_services_tables_consolidated.csv'
df = pd.read_csv(file_df)
df


In [ ]:
# def get_icd_table(df):
#     icd_table = df[df['id'].between(100, 200)].loc[:, 'name']
#     return icd_table


In [ ]:
# get_icd_table(df)

In [ ]:

file_df= r'C:\Users\jchri\PycharmProjects\dash_website\src\app\data\corp_services_tables_consolidated.csv'
df1 = pd.read_csv(file_df)
df1.info()


In [ ]:

class CorporateTables:
    def __init__(self, dataframe):
        self.corp_table = dataframe

    def get_icd_table(self):
        table = self.corp_table[self.corp_table['id'].between(100, 200)].loc[:,'name']
        return table


In [ ]:
services_table = CorporateTables(df1)
icd_table = services_table.get_icd_table()
icd_table


In [ ]:
# injury_disease
# specialty
# facility

# CORPORATE SERVICES SECTION

INJURY DISEASE

In [ ]:
with app.app_context():
    stmt = db.select(InjuryDiseaseSummary.name, InjuryDiseaseSummary.claim_count).order_by(InjuryDiseaseSummary.claim_count)
    result = db.session.execute(stmt)
    # for r in result:
    #     print(r)
    icd_count = pd.DataFrame(result)
    # icd_count

INJURY_DISEASE TABLE

In [ ]:


with app.app_context():
    icd_count = (db.session.execute(db.select(InjuryDiseaseSummary.name, InjuryDiseaseSummary.claim_count,
                                              InjuryDiseaseSummary.claim_paid,
                                              InjuryDiseaseSummary.color_code)
                                    .order_by(InjuryDiseaseSummary.claim_count))
                 )

    table = pd.DataFrame(icd_count)
    table = table.nlargest(10,'claim_count')
    # icd_count_chart

    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=table['name'],
            y=table['claim_count'],
            marker_color=table['color_code']
        )
    )

    table2 = table.sort_values('claim_paid')
    # table2


    fig2 = go.Figure()
    fig2.add_trace(
        go.Bar(
            x=table2['name'],
            y=table2['claim_paid'],
            marker_color=table2['color_code']
        )
    )

SPECIALTY TABLE

In [ ]:
with app.app_context():
    specialty_table = (db.session.execute(db.select(SpecialtySummary.name, SpecialtySummary.claim_count,
                                              SpecialtySummary.claim_paid,
                                              SpecialtySummary.color_code)
                                    .order_by(SpecialtySummary.claim_count))
                 )

    table = pd.DataFrame(specialty_table)
    table = table.nlargest(10,'claim_count')
    # table

    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=table['name'],
            y=table['claim_count'],
            marker_color=table['color_code']
        )
    )

    # table2 = table.sort_values('claim_paid')
    # # table2
    #
    #
    # fig2 = go.Figure()
    # fig2.add_trace(
    #     go.Bar(
    #         x=table2['name'],
    #         y=table2['claim_paid'],
    #         marker_color=table2['color_code']
    #     )
    # )